## Partie 0 : Introduction aux Bases de Données Time-Series

### 0.1 Qu'est-ce qu'une donnée time-series ?

Une donnée time-series (série temporelle) est une séquence de points de données indexés dans l'ordre temporel. Exemples courants :

- Capteurs IoT : température, pression, humidité mesurées toutes les secondes
- Tracking d'animaux : positions GPS avec timestamp (notre cas)
- Finance : cours de bourse, transactions bancaires
- Monitoring IT : métriques serveur (CPU, RAM, réseau)

Caractéristiques clés :
- Chaque point a un timestamp précis
- Les données arrivent généralement dans l'ordre chronologique
- Souvent volumineuses (millions de points)
- Les requêtes portent fréquemment sur des intervalles de temps

### 0.2 Pourquoi InfluxDB plutôt que MongoDB ou MySQL ?

| Critère | MySQL | MongoDB | InfluxDB |
|---------|-------|---------|----------|
| Structure | Tables relationnelles | Documents JSON | Mesures + tags + fields + time |
| Indexation temporelle | Index B-tree standard | Index classique | Index temporel natif (TSM) |
| Agrégations temporelles | Lent | Acceptable | Très rapide |
| Compression | Faible | Moyenne | Très élevée (10-20x) |
| Cas d'usage | E-commerce, CRM | Applications web | **Monitoring, IoT, tracking** |

Pour notre dataset de migration d'oiseaux :
- 89 867 points temporels → structure time-series
- Requêtes fréquentes par intervalle de temps
- Besoin d'agrégations temporelles (distance parcourue par jour)
- Compression importante (données géographiques répétitives)

### 0.3 Concepts Clés d'InfluxDB

InfluxDB organise les données différemment des bases relationnelles ou documents :

```
┌─────────────────────────────────────────────────────────────┐
│ MEASUREMENT (table)                                          │
├─────────────────────────────────────────────────────────────┤
│ Tags (indexés)          │ Fields (valeurs)    │ Timestamp   │
│ ======================= │ =================== │ =========== │
│ bird_id: "91732A"       │ latitude: 61.24     │ 2009-05-27  │
│ tag_id: 91732           │ longitude: 24.58    │ 14:00:00    │
│                         │ vegetation: 0.96    │             │
└─────────────────────────────────────────────────────────────┘
```

**1. Measurement** : nom de la "table" (ex: `bird_migration`)

**2. Tags** : métadonnées indexées
   - Chaînes de caractères uniquement
   - Cardinalité modérée recommandée (< 100k valeurs uniques)
   - Utilisés pour filtrer : `WHERE bird_id = "91732A"`
   - Exemples : bird_id, sensor_type, region

**3. Fields** : valeurs mesurées
   - Peuvent être de tout type (float, int, string, bool)
   - Non indexés → ne pas filtrer dessus si possible
   - Exemples : latitude, longitude, température, vitesse

**4. Timestamp** : horodatage du point (nanoseconde precision)

Règle d'or pour choisir tag vs field :
- Tag si vous filtrez souvent dessus (`WHERE ...`)
- Field si c'est une valeur mesurée/calculée
- Tag seulement si cardinalité raisonnable (éviter timestamps, IDs uniques)

## Partie 1 : Installation et Configuration

### 1.1 Importation des bibliothèques nécessaires

In [52]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# InfluxDB
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

# Kaggle
import kagglehub

# Visualisation
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import HeatMap, MarkerCluster

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

### 1.2 Configuration InfluxDB

Configuration requise :
- **URL** : adresse du serveur InfluxDB (ici dans Docker : `http://influxdb2:8086`)
- **Token** : clé d'authentification (admin-token configuré dans docker-compose)
- **Organisation** : espace de travail logique
- **Bucket** : "base de données" où stocker les données

In [53]:
# Configuration InfluxDB
INFLUX_URL = "http://influxdb2:8086" 
INFLUX_TOKEN = "admin-token"
INFLUX_ORG = "fil-A3-back-bigData"
INFLUX_BUCKET = "animal-tracking"  

# Connexion au client
client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

print("Client InfluxDB connecté")
print(f"  Bucket: {INFLUX_BUCKET}")
print(f"  Organisation: {INFLUX_ORG}")

# Vérification de la connexion
try:
    if client.ping():
        print("Serveur InfluxDB répond")
    else:
        print("Serveur ne répond pas")
except Exception as e:
    print(f"Erreur de connexion: {e}")

Client InfluxDB connecté
  Bucket: animal-tracking
  Organisation: fil-A3-back-bigData
Serveur InfluxDB répond


## Partie 2 : Chargement et Exploration des Données

Comprendre la structure des données et identifier ce qui en fait une série temporelle.

In [54]:
import os

# Download the Movebank animal tracking dataset
path = "pulkit8595/movebank-animal-tracking"
print("Downloading dataset from Kaggle...")
dataset_path = kagglehub.dataset_download(path)
print(f"Dataset downloaded to: {dataset_path}")

# List files in the dataset
print("\nFiles in dataset:")
for file in os.listdir(dataset_path):
    file_size = os.path.getsize(os.path.join(dataset_path, file)) / 1024  # KB
    print(f"  - {file} ({file_size:.2f} KB)")

Dataset downloaded to: /home/jovyan/.cache/kagglehub/datasets/pulkit8595/movebank-animal-tracking/versions/1

Files in dataset:
  - migration_original.csv (21766.21 KB)


In [55]:
df = pd.read_csv(os.path.join(dataset_path, "migration_original.csv"))

print("Dataset loaded into DataFrame")
print(f"DataFrame shape: {df.shape}")
df.head()

Dataset loaded into DataFrame
DataFrame shape: (89867, 15)


,event-id,visible,timestamp,location-long,location-lat,manually-marked-outlier,visible.1,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name,ECMWF Interim Full Daily Invariant Low Vegetation Cover,NCEP NARR SFC Vegetation at Surface,ECMWF Interim Full Daily Invariant High Vegetation Cover
0,1082620685,True,2009-05-27 14:00:00.000,24.58617,61.24783,NaN,True,gps,Larus fuscus,91732,91732A,Navigation experiments in lesser black-backed ...,0.039229,NaN,0.960771
1,1082620686,True,2009-05-27 20:00:00.000,24.58217,61.23267,NaN,True,gps,Larus fuscus,91732,91732A,Navigation experiments in lesser black-backed ...,0.040803,NaN,0.959197
2,1082620687,True,2009-05-28 05:00:00.000,24.53133,61.18833,NaN,True,gps,Larus fuscus,91732,91732A,Navigation experiments in lesser black-backed ...,0.052201,NaN,0.947799
3,1082620688,True,2009-05-28 08:00:00.000,24.58200,61.23283,NaN,True,gps,Larus fuscus,91732,91732A,Navigation experiments in lesser black-backed ...,0.040818,NaN,0.959182
4,1082620689,True,2009-05-28 14:00:00.000,24.58250,61.23267,NaN,True,gps,Larus fuscus,91732,91732A,Navigation experiments in lesser black-backed ...,0.040753,NaN,0.959247


In [56]:
# Informations sur le dataset
print("Informations sur le dataset:")
print(df.info())
print("\n Statistiques descriptives:")
print(df.describe())
print("\n Valeurs manquantes:")
print(df.isnull().sum())

Informations sur le dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89867 entries, 0 to 89866
Data columns (total 15 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   event-id                                                  89867 non-null  int64  
 1   visible                                                   89867 non-null  bool   
 2   timestamp                                                 89867 non-null  object 
 3   location-long                                             89867 non-null  float64
 4   location-lat                                              89867 non-null  float64
 5   manually-marked-outlier                                   0 non-null      float64
 6   visible.1                                                 89867 non-null  bool   
 7   sensor-type                                               89867 non-null  

### 2.1 Analyse de cardinalité (crucial pour le design du schéma)

La cardinalité (nombre de valeurs distinctes) détermine si une colonne doit être un tag ou un field dans InfluxDB :
- Cardinalité basse (< 100 valeurs) → Bon candidat pour tag
- Cardinalité moyenne (100-100k valeurs) → Acceptable comme tag si filtrage fréquent
- Cardinalité haute (> 100k valeurs) → NE PAS mettre en tag (explosion des index)

In [57]:
# Analyse de cardinalité pour chaque colonne
print("=" * 70)
print("ANALYSE DE CARDINALITÉ".center(70))
print("=" * 70)
print(f"{'Colonne':<50} {'Cardinalité':>15}")
print("-" * 70)

for col in df.columns:
    cardinality = df[col].nunique()
    total = len(df)
    percentage = (cardinality / total) * 100
    
    # Indicateur selon le type de cardinalité
    if cardinality == 1:
        indicator = "CONSTANTE"
    elif cardinality < 100:
        indicator = "BASSE (bon tag)"
    elif cardinality < 10000:
        indicator = "MOYENNE (tag ok)"
    elif cardinality == total:
        indicator = "UNIQUE (PK)"
    else:
        indicator = "HAUTE (field)"
    
    print(f"{col:<50} {cardinality:>10,} ({percentage:>5.1f}%)  {indicator}")

print("=" * 70)

                        ANALYSE DE CARDINALITÉ                        
Colonne                                                Cardinalité
----------------------------------------------------------------------
event-id                                               89,867 (100.0%)  UNIQUE (PK)
visible                                                     1 (  0.0%)  CONSTANTE
timestamp                                              16,604 ( 18.5%)  HAUTE (field)
location-long                                          35,041 ( 39.0%)  HAUTE (field)
location-lat                                           34,252 ( 38.1%)  HAUTE (field)
manually-marked-outlier                                     0 (  0.0%)  BASSE (bon tag)
visible.1                                                   1 (  0.0%)  CONSTANTE
sensor-type                                                 1 (  0.0%)  CONSTANTE
individual-taxon-canonical-name                             1 (  0.0%)  CONSTANTE
tag-local-identifier             

### 2.2 Visualisation géographique rapide

Où se trouvent les oiseaux ? Migration visible ?

In [59]:
# Préparer les données
sample_df = df.iloc[::5].copy()
sample_df['date'] = pd.to_datetime(sample_df['timestamp'])
sample_df['bird_id'] = sample_df['individual-local-identifier']


# Visualisation des trajectoires d'oiseaux
fig = px.scatter_geo(sample_df,
                     lat='location-lat',
                     lon='location-long',
                     color='bird_id',
                     hover_data=['date', 'bird_id'],
                     title='Migration Patterns - Bird Tracking Data',
                     projection='natural earth')

fig.update_layout(
    height=600,
    showlegend=True,
    geo=dict(
        showland=True,
        landcolor='rgb(243, 243, 243)',
        coastlinecolor='rgb(204, 204, 204)',
    )
)

fig.show()

# Statistiques par oiseau
print(f"Dataset échantillonné: {len(sample_df):,} points")
print(f"Nombre d'oiseaux: {sample_df['bird_id'].nunique()}")
print(f"Période: {sample_df['date'].min()} → {sample_df['date'].max()}")

Dataset échantillonné: 17,974 points
Nombre d'oiseaux: 126
Période: 2009-05-25 00:08:00 → 2015-08-26 09:00:00


## Partie 3 : Nettoyage des Données

Préparer les données pour InfluxDB en supprimant les colonnes inutiles.

Stratégie de nettoyage :

1. Supprimer les colonnes 100% vides (aucune valeur utile)
   - Exemple : `manually-marked-outlier`, `NCEP NARR SFC Vegetation at Surface`
   - Ces colonnes ajouteraient des fields inutiles dans InfluxDB

2. Garder les colonnes redondantes mais utiles
   - `visible` et `visible.1` semblent dupliquées mais gardons-les pour l'instant
   - Si vraiment identiques, on pourra supprimer après vérification

3. Renommer les colonnes pour la lisibilité
   - `ECMWF Interim Full Daily...` → `vegetation_cover_low/high`
   - Noms courts = queries Flux plus lisibles

In [60]:
# Supprimer les colonnes 100% vides
columns_to_drop = df.columns[df.isnull().all()].tolist()
df_clean = df.drop(columns=columns_to_drop)

# Renommer les colonnes de végétation pour plus de clarté
df_clean = df_clean.rename(columns={
    'ECMWF Interim Full Daily Invariant Low Vegetation Cover': 'vegetation_cover_low',
    'ECMWF Interim Full Daily Invariant High Vegetation Cover': 'vegetation_cover_high'
})

print(f"Colonnes supprimées (100% vides): {len(columns_to_drop)}")
print(f"Colonnes renommées: vegetation_cover_low/high")
print(f"Dataset nettoyé: {df_clean.shape[0]:,} lignes × {df_clean.shape[1]} colonnes")

# Vérifier valeurs manquantes restantes
null_counts = df_clean.isnull().sum()
remaining_nulls = null_counts[null_counts > 0]
if len(remaining_nulls) > 0:
    print(f"\nValeurs manquantes restantes:")
    for col, count in remaining_nulls.items():
        print(f"  {col}: {count:,} ({count/len(df_clean)*100:.1f}%)")

Colonnes supprimées (100% vides): 2
Colonnes renommées: vegetation_cover_low/high
Dataset nettoyé: 89,867 lignes × 13 colonnes


## Partie 4 : Design du Schéma et Insertion dans InfluxDB

Comprendre comment choisir entre tags et fields, puis insérer les données efficacement.

### 4.1 Décisions de Design : Tags vs Fields

Rappel de la règle :
- Tags = filtres fréquents + cardinalité raisonnable
- Fields = valeurs mesurées + pas de filtrage

Analysons chaque colonne :

## MATRICE DE DÉCISION : TAGS vs FIELDS

| Colonne | Cardinalité | Type | Décision | Raison |
|---------|-------------|------|----------|--------|
| `timestamp` | 16,604 | ⏰ **TIMESTAMP** | Timestamp InfluxDB | Index temporel principal |
| `individual-local-identifier` | 126 | 🏷️ **TAG** | Tag | Filtrage fréquent par oiseau, ~199 valeurs (cardinalité ok) |
| `tag-local-identifier` | 126 | 🏷️ **TAG** | Tag | Filtrage par device, ~199 valeurs (cardinalité ok) |
| `sensor-type` | 1 | 📊 **FIELD** | Field | ⚠️ CARDINALITÉ = 1 ! Index inutile, gaspillage mémoire |
| `individual-taxon-canonical-name` | 1 | 📊 **FIELD** | Field | Cardinalité = 1 (tous *Larus fuscus*), même raison |
| `study-name` | 1 | 📊 **FIELD** | Field | Cardinalité = 1, pas de filtrage nécessaire |
| `event-id` | 89,867 | 📊 **FIELD** | Field | ⚠️ Cardinalité = 89,867 (unique), **jamais en tag!** |
| `location-lat` | 34,252 | 📊 **FIELD** | Field | Valeur mesurée (coordonnée GPS) |
| `location-long` | 35,041 | 📊 **FIELD** | Field | Valeur mesurée (coordonnée GPS) |
| `vegetation_cover_low` | 56,794 | 📊 **FIELD** | Field | Valeur mesurée (donnée environnementale) |
| `vegetation_cover_high` | 51,905 | 📊 **FIELD** | Field | Valeur mesurée (donnée environnementale) |
| `visible / visible.1` | 1 | 📊 **FIELD** | Field | Métadonnée booléenne, pas de filtrage |

---

### 📋 RÉSUMÉ DU SCHÉMA OPTIMISÉ

| Catégorie | Colonnes |
|-----------|----------|
| **🏷️ Tags (2)** | `individual-local-identifier`, `tag-local-identifier` |
| **📊 Fields (9)** | `location-lat`, `location-long`, `event-id`, `vegetation_cover_*`, `sensor-type`, `study-name`, `individual-taxon-canonical-name`, `visible` |
| **⏰ Timestamp** | `timestamp` |

**Ratio Tags/Fields : 2:9** ✅ (Optimal pour notre cas d'usage)

### 4.2 Anti-Patterns à Éviter

NE PAS mettre en tags :
- Colonnes avec cardinalité = 1 (constantes) → gaspillage mémoire
- Colonnes avec haute cardinalité (> 100k) → explosion des index
- Timestamps → utiliser le timestamp InfluxDB
- Floats avec haute cardinalité → utiliser fields

Notre schéma optimisé : 2 tags (~199 valeurs chacun), reste en fields

### 4.3 Insertion des Données

In [61]:
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])

print(f"Insertion de {len(df_clean):,} records dans InfluxDB...")
print(f"Tags: individual-local-identifier, tag-local-identifier")
print(f"Fields: {df_clean.shape[1] - 3} colonnes")

import time
start_time = time.time()

try:
    write_api.write(
        bucket=INFLUX_BUCKET, 
        org=INFLUX_ORG, 
        record=df_clean, 
        data_frame_measurement_name='bird_migration',
        data_frame_tag_columns=[
            'individual-local-identifier',
            'tag-local-identifier',
        ],
        data_frame_timestamp_column='timestamp'
    )
    
    elapsed = time.time() - start_time
    print(f"\nInsertion réussie en {elapsed:.2f}s ({len(df_clean)/elapsed:.0f} points/s)")
    
except Exception as e:
    print(f"Erreur: {e}")
    raise

Insertion de 89,867 records dans InfluxDB...
Tags: individual-local-identifier, tag-local-identifier
Fields: 10 colonnes

Insertion réussie en 1.44s (62205 points/s)


### 4.4 Vérification de l'Insertion

Vérifions que les données ont bien été insérées dans InfluxDB en exécutant une requête simple.

In [62]:
# Vérification simple : récupérer quelques points
verification_query = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> limit(n: 10)
'''

result = query_api.query_data_frame(verification_query, org=INFLUX_ORG)

# Traiter le résultat
if isinstance(result, list) and len(result) > 0:
    result_df = pd.concat(result, ignore_index=True)
    print(f"Données trouvées: {len(result_df)} lignes")
    print(result_df.head())
elif isinstance(result, pd.DataFrame) and len(result) > 0:
    print(f"Données trouvées: {len(result)} lignes")
    print(result.head())
else:
    print("Aucune donnée trouvée")

Données trouvées: 12350 lignes
    result  table                    _start                            _stop  \
0  _result      0 1970-01-01 00:00:00+00:00 2025-11-20 19:19:45.828103+00:00   
1  _result      0 1970-01-01 00:00:00+00:00 2025-11-20 19:19:45.828103+00:00   
2  _result      0 1970-01-01 00:00:00+00:00 2025-11-20 19:19:45.828103+00:00   
3  _result      0 1970-01-01 00:00:00+00:00 2025-11-20 19:19:45.828103+00:00   
4  _result      0 1970-01-01 00:00:00+00:00 2025-11-20 19:19:45.828103+00:00   

                      _time      _value    _field    _measurement  \
0 2009-05-27 14:00:00+00:00  1082620685  event-id  bird_migration   
1 2009-05-27 20:00:00+00:00  1082620686  event-id  bird_migration   
2 2009-05-28 05:00:00+00:00  1082620687  event-id  bird_migration   
3 2009-05-28 08:00:00+00:00  1082620688  event-id  bird_migration   
4 2009-05-28 14:00:00+00:00  1082620689  event-id  bird_migration   

  individual-local-identifier tag-local-identifier  
0                   

## Partie 5 : Langage de Requêtes Flux - Construction Progressive

Maîtriser Flux en construisant des requêtes de plus en plus complexes.

In [ ]:
# Helper function pour afficher les résultats Flux proprement
def display_flux_result(result, title="Résultat"):
    """Affiche les résultats d'une requête Flux de manière lisible"""
    if isinstance(result, list):
        if len(result) > 0:
            result_df = pd.concat(result, ignore_index=True)
            print(f"{title}: {len(result_df)} lignes")
            display(result_df.head(10))
            return result_df
        else:
            print(f"{title}: Aucune donnée")
            return None
    else:
        print(f"{title}: {len(result)} lignes")
        if len(result) > 0:
            display(result.head(10))
        return result

def process_flux_result(result):
    """Convertit le résultat Flux en DataFrame"""
    if isinstance(result, list) and len(result) > 0:
        return pd.concat(result, ignore_index=True)
    return result if isinstance(result, pd.DataFrame) else None

### 5.1 Requête de Base : Récupérer des Données

Anatomie d'une requête Flux :
```
from(bucket: "nom")              ← Source des données
  |> range(start: xxx, stop: yyy)  ← Filtre temporel (OBLIGATOIRE)
  |> filter(fn: (r) => ...)        ← Filtres supplémentaires
  |> limit(n: 10)                  ← Limiter le nombre de résultats
```

Pipeline : Chaque opérateur `|>` passe les données à l'opérateur suivant

In [63]:
# Requête 1: Récupérer les 100 premiers points
query_basic = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)  
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> limit(n: 100)
'''

print("=" * 80)
print("REQUÊTE 1: RÉCUPÉRATION BASIQUE (100 premiers points)".center(80))
print("=" * 80)
print(query_basic)
print()

result_basic = query_api.query_data_frame(query_basic, org=INFLUX_ORG)
df_basic = display_flux_result(result_basic, "Requête basique")

             REQUÊTE 1: RÉCUPÉRATION BASIQUE (100 premiers points)              

from(bucket: "animal-tracking")
  |> range(start: 0)  
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> limit(n: 100)


Requête basique: 101880 lignes


,result,table,_start,_stop,_time,_value,_field,_measurement,individual-local-identifier,tag-local-identifier
0,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-27 14:00:00+00:00,1082620685,event-id,bird_migration,91732A,91732
1,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-27 20:00:00+00:00,1082620686,event-id,bird_migration,91732A,91732
2,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-28 05:00:00+00:00,1082620687,event-id,bird_migration,91732A,91732
3,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-28 08:00:00+00:00,1082620688,event-id,bird_migration,91732A,91732
4,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-28 14:00:00+00:00,1082620689,event-id,bird_migration,91732A,91732
5,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-28 20:00:00+00:00,1082620690,event-id,bird_migration,91732A,91732
6,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-29 05:00:00+00:00,1082620691,event-id,bird_migration,91732A,91732
7,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-29 08:00:00+00:00,1082620692,event-id,bird_migration,91732A,91732
8,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-29 14:00:00+00:00,1082620693,event-id,bird_migration,91732A,91732
9,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:07.885660+00:00,2009-05-29 20:00:00+00:00,1082620694,event-id,bird_migration,91732A,91732


### 5.2 Filtrage par Tag : Requêtes Efficaces

Pourquoi filtrer sur les tags : Index optimisés → queries ultra-rapides

Équivalence :
- Flux : `filter(fn: (r) => r["individual-local-identifier"] == "91732A")`
- SQL : `WHERE individual_local_identifier = '91732A'`
- Pandas : `df[df['individual-local-identifier'] == '91732A']`

In [64]:
bird_id = "91732A"

query_by_bird = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["individual-local-identifier"] == "{bird_id}")
  |> limit(n: 50)
'''

print("=" * 80)
print(f"REQUÊTE 2: FILTRAGE PAR TAG (oiseau {bird_id})".center(80))
print("=" * 80)

result_by_bird = query_api.query_data_frame(query_by_bird, org=INFLUX_ORG)
df_by_bird = display_flux_result(result_by_bird, f"Données pour {bird_id}")

                  REQUÊTE 2: FILTRAGE PAR TAG (oiseau 91732A)                   
Données pour 91732A: 500 lignes


,result,table,_start,_stop,_time,_value,_field,_measurement,individual-local-identifier,tag-local-identifier
0,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-27 14:00:00+00:00,1082620685,event-id,bird_migration,91732A,91732
1,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-27 20:00:00+00:00,1082620686,event-id,bird_migration,91732A,91732
2,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-28 05:00:00+00:00,1082620687,event-id,bird_migration,91732A,91732
3,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-28 08:00:00+00:00,1082620688,event-id,bird_migration,91732A,91732
4,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-28 14:00:00+00:00,1082620689,event-id,bird_migration,91732A,91732
5,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-28 20:00:00+00:00,1082620690,event-id,bird_migration,91732A,91732
6,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-29 05:00:00+00:00,1082620691,event-id,bird_migration,91732A,91732
7,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-29 08:00:00+00:00,1082620692,event-id,bird_migration,91732A,91732
8,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-29 14:00:00+00:00,1082620693,event-id,bird_migration,91732A,91732
9,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:22:46.854890+00:00,2009-05-29 20:00:00+00:00,1082620694,event-id,bird_migration,91732A,91732


### 5.3 Filtrage Temporel : La Force des Time-Series DB

C'est ici que InfluxDB brille : requêtes temporelles ultra-optimisées

Exemple : Trouver les données de migration estivale (juin-août 2009)

In [65]:
query_summer = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 2009-06-01T00:00:00Z, stop: 2009-08-31T23:59:59Z)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> limit(n: 100)
'''

print("=" * 80)
print("REQUÊTE 3: FILTRAGE TEMPOREL (été 2009)".center(80))
print("=" * 80)

result_summer = query_api.query_data_frame(query_summer, org=INFLUX_ORG)
df_summer = display_flux_result(result_summer, "Données été 2009")

                    REQUÊTE 3: FILTRAGE TEMPOREL (été 2009)                     
Données été 2009: 7539 lignes


,result,table,_start,_stop,_time,_value,_field,_measurement,individual-local-identifier,tag-local-identifier
0,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-01 05:00:00+00:00,61.23300,location-lat,bird_migration,91732A,91732
1,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-02 05:00:00+00:00,61.22933,location-lat,bird_migration,91732A,91732
2,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-02 14:00:00+00:00,61.21700,location-lat,bird_migration,91732A,91732
3,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-02 20:00:00+00:00,61.19000,location-lat,bird_migration,91732A,91732
4,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-03 05:00:00+00:00,61.23317,location-lat,bird_migration,91732A,91732
5,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-03 08:00:00+00:00,61.24783,location-lat,bird_migration,91732A,91732
6,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-03 14:00:00+00:00,61.24783,location-lat,bird_migration,91732A,91732
7,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-03 20:00:00+00:00,61.26983,location-lat,bird_migration,91732A,91732
8,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-04 05:00:00+00:00,61.26833,location-lat,bird_migration,91732A,91732
9,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-04 08:00:00+00:00,61.24767,location-lat,bird_migration,91732A,91732


### 5.4 Restructuration avec Pivot : Transformer Fields en Colonnes

Problème : InfluxDB retourne 1 ligne par field → difficile à analyser

Solution : `pivot()` transforme les fields en colonnes (comme un DataFrame)

In [66]:
query_pivot = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["individual-local-identifier"] == "91732A")
  |> filter(fn: (r) => r["_field"] == "location-lat" or r["_field"] == "location-long")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> limit(n: 20)
'''

print("=" * 80)
print("REQUÊTE 4: PIVOT (fields → colonnes)".center(80))
print("=" * 80)

result_pivot = query_api.query_data_frame(query_pivot, org=INFLUX_ORG)
df_pivot = display_flux_result(result_pivot, "Données pivotées")

                      REQUÊTE 4: PIVOT (fields → colonnes)                      
Données pivotées: 20 lignes


,result,table,_start,_stop,_time,_measurement,individual-local-identifier,tag-local-identifier,location-lat,location-long
0,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-27 14:00:00+00:00,bird_migration,91732A,91732,61.24783,24.58617
1,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-27 20:00:00+00:00,bird_migration,91732A,91732,61.23267,24.58217
2,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-28 05:00:00+00:00,bird_migration,91732A,91732,61.18833,24.53133
3,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-28 08:00:00+00:00,bird_migration,91732A,91732,61.23283,24.58200
4,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-28 14:00:00+00:00,bird_migration,91732A,91732,61.23267,24.58250
5,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-28 20:00:00+00:00,bird_migration,91732A,91732,61.24767,24.58617
6,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-29 05:00:00+00:00,bird_migration,91732A,91732,61.24767,24.58600
7,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-29 08:00:00+00:00,bird_migration,91732A,91732,61.24767,24.58617
8,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-29 14:00:00+00:00,bird_migration,91732A,91732,61.24750,24.58650
9,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:01.551130+00:00,2009-05-29 20:00:00+00:00,bird_migration,91732A,91732,61.23883,24.56967


### 5.5 Agrégation : Compter, Grouper, Analyser

Opérations d'agrégation courantes :
- `count()` : nombre de points
- `mean()` : moyenne
- `sum()` : somme
- `max()` / `min()` : valeurs extrêmes
- `group()` : grouper par tag(s)

Exemple : Compter le nombre de tracking points par oiseau

In [67]:
query_count = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> group(columns: ["individual-local-identifier"])
  |> count()
  |> sort(columns: ["_value"], desc: true)
  |> limit(n: 20)
'''

print("=" * 80)
print("REQUÊTE 5: AGRÉGATION (count par oiseau)".center(80))
print("=" * 80)

result_count = query_api.query_data_frame(query_count, org=INFLUX_ORG)
df_count = display_flux_result(result_count, "Comptage par oiseau")

                    REQUÊTE 5: AGRÉGATION (count par oiseau)                    
Comptage par oiseau: 126 lignes


,result,table,_start,_stop,_value,individual-local-identifier
0,_result,0,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,1970,91732A
1,_result,1,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,44,91733A
2,_result,2,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,1060,91734A
3,_result,3,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,477,91735A
4,_result,4,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,1039,91737A
5,_result,5,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,424,91738A
6,_result,6,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,2738,91739A
7,_result,7,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,335,91740A
8,_result,8,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,415,91741A
9,_result,9,1970-01-01 00:00:00+00:00,2025-11-20 19:25:57.110710+00:00,1006,91742A


### 5.6 Window Functions : Agrégation Temporelle

Cas d'usage : Calculer des moyennes par jour/heure/mois

`aggregateWindow()` divise le temps en fenêtres et agrège chacune séparément

In [68]:
query_window = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 2009-05-01T00:00:00Z, stop: 2009-05-31T23:59:59Z)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "vegetation_cover_high")
  |> aggregateWindow(every: 1d, fn: mean, createEmpty: false)
  |> limit(n: 31)
'''

print("=" * 80)
print("REQUÊTE 6: WINDOW FUNCTION (moyenne journalière)".center(80))
print("=" * 80)

result_window = query_api.query_data_frame(query_window, org=INFLUX_ORG)
df_window = display_flux_result(result_window, "Moyenne quotidienne végétation")

                REQUÊTE 6: WINDOW FUNCTION (moyenne journalière)                
Moyenne quotidienne végétation: 113 lignes


,result,table,_start,_stop,_time,_value,_field,_measurement,individual-local-identifier,tag-local-identifier
0,_result,0,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-28 00:00:00+00:00,0.959984,vegetation_cover_high,bird_migration,91732A,91732
1,_result,0,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-29 00:00:00+00:00,0.956747,vegetation_cover_high,bird_migration,91732A,91732
2,_result,0,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-30 00:00:00+00:00,0.960004,vegetation_cover_high,bird_migration,91732A,91732
3,_result,0,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-31 00:00:00+00:00,0.960787,vegetation_cover_high,bird_migration,91732A,91732
4,_result,0,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-31 23:59:59+00:00,0.951123,vegetation_cover_high,bird_migration,91732A,91732
5,_result,1,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-26 00:00:00+00:00,0.900641,vegetation_cover_high,bird_migration,91734A,91734
6,_result,1,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-27 00:00:00+00:00,0.903757,vegetation_cover_high,bird_migration,91734A,91734
7,_result,1,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-28 00:00:00+00:00,0.902672,vegetation_cover_high,bird_migration,91734A,91734
8,_result,1,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-29 00:00:00+00:00,0.905985,vegetation_cover_high,bird_migration,91734A,91734
9,_result,1,2009-05-01 00:00:00+00:00,2009-05-31 23:59:59+00:00,2009-05-30 00:00:00+00:00,0.908527,vegetation_cover_high,bird_migration,91734A,91734


## Partie 6 : Visualisations - Démonstration (OPTIONNEL)

Note : Cette partie est une démonstration optionnelle pour montrer ce qu'on peut faire avec les données. L'objectif principal du TP est d'apprendre InfluxDB (schema design, Flux queries), pas la visualisation de données.

Montrer rapidement quelques visualisations possibles des patterns de migration.

### 6.1 Carte Interactive des Routes de Migration

Visualiser les trajectoires des 3 oiseaux les plus trackés sur une carte interactive avec Folium.

In [ ]:
query_top_birds = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> group(columns: ["individual-local-identifier"])
  |> count()
  |> group()
  |> sort(columns: ["_value"], desc: true)
  |> limit(n: 3)
'''

result_top = query_api.query_data_frame(query_top_birds, org=INFLUX_ORG)
df_top = process_flux_result(result_top)

# S'assurer qu'on a bien seulement 3 oiseaux
if df_top is not None and len(df_top) > 3:
    df_top = df_top.head(3)

top_bird_ids = df_top['individual-local-identifier'].tolist()

print(f"Top 3 oiseaux: {', '.join(top_bird_ids)}")

m = folium.Map(location=[50, 15], zoom_start=4, tiles='OpenStreetMap')
colors = ['red', 'blue', 'green']

for idx, bird_id in enumerate(top_bird_ids):
    query_traj = f'''
    from(bucket: "{INFLUX_BUCKET}")
      |> range(start: 0)
      |> filter(fn: (r) => r["_measurement"] == "bird_migration")
      |> filter(fn: (r) => r["individual-local-identifier"] == "{bird_id}")
      |> filter(fn: (r) => r["_field"] == "location-lat" or r["_field"] == "location-long")
      |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
      |> sort(columns: ["_time"])
    '''
    
    result_traj = query_api.query_data_frame(query_traj, org=INFLUX_ORG)
    df_traj = process_flux_result(result_traj)
    
    if df_traj is not None and len(df_traj) > 0:
        # Filtrer les lignes avec des valeurs NaN
        df_traj = df_traj.dropna(subset=['location-lat', 'location-long'])
        
        if len(df_traj) > 0:  # Vérifier à nouveau après le dropna
            points = list(zip(df_traj['location-lat'], df_traj['location-long']))
            
            if len(points) > 0:  # Vérifier que points n'est pas vide
                from folium.plugins import AntPath
                AntPath(points, color=colors[idx], weight=2.5, opacity=0.8, delay=800,
                        popup=f"<b>{bird_id}</b><br>{len(points)} points").add_to(m)
                
                folium.Marker(points[0], popup=f"Départ: {bird_id}",
                             icon=folium.Icon(color=colors[idx], icon='play')).add_to(m)
                folium.Marker(points[-1], popup=f"Fin: {bird_id}",
                             icon=folium.Icon(color=colors[idx], icon='stop')).add_to(m)
            else:
                print(f"Aucun point valide pour l'oiseau {bird_id}")

display(m)

### 6.2 Time-Series: Latitude au Fil du Temps

Visualiser le pattern de migration saisonnière d'un oiseau (latitude vs temps).

In [ ]:
main_bird = top_bird_ids[0]

query_lat_time = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["individual-local-identifier"] == "{main_bird}")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> sort(columns: ["_time"])
'''

result_lat = query_api.query_data_frame(query_lat_time, org=INFLUX_ORG)
df_lat = process_flux_result(result_lat)

if df_lat is not None and len(df_lat) > 0:
    df_lat['_time'] = pd.to_datetime(df_lat['_time'])
    
    fig, ax = plt.subplots(figsize=(16, 6))
    ax.plot(df_lat['_time'], df_lat['_value'], linewidth=1.5, color='#2E86AB', alpha=0.8)
    
    years = df_lat['_time'].dt.year.unique()
    for year in years:
        summer_start = pd.Timestamp(f'{year}-06-01')
        summer_end = pd.Timestamp(f'{year}-08-31')
        ax.axvspan(summer_start, summer_end, alpha=0.2, color='yellow', 
                   label='Été' if year == years[0] else '')
        
        winter_start = pd.Timestamp(f'{year}-12-01')
        winter_end = pd.Timestamp(f'{year+1}-02-28')
        ax.axvspan(winter_start, winter_end, alpha=0.2, color='lightblue',
                   label='Hiver' if year == years[0] else '')
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Latitude (°)', fontsize=12)
    ax.set_title(f'Pattern de Migration: {main_bird}', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right')
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("Aucune donnée")

### 6.3 Heatmap: Densité des Points de Tracking en Fonction du Temps

Visualiser l'évolution temporelle des zones de présence des oiseaux (migration saisonnière).

In [ ]:
query_all_points = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat" or r["_field"] == "location-long")
  |> sample(n: 1000)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

result_all = query_api.query_data_frame(query_all_points, org=INFLUX_ORG)
df_all = process_flux_result(result_all)

if df_all is not None and len(df_all) > 0:
    from folium.plugins import HeatMap
    
    # Filtrer les lignes avec des valeurs NaN dans les coordonnées
    df_all_clean = df_all.dropna(subset=['location-lat', 'location-long'])
    
    if len(df_all_clean) > 0:
        m_heat = folium.Map(location=[45, 15], zoom_start=4, tiles='CartoDB positron')
        heat_data = [[row['location-lat'], row['location-long']] for idx, row in df_all_clean.iterrows()]
        
        HeatMap(heat_data, radius=8, blur=10, max_zoom=6,
                gradient={0.4: 'blue', 0.65: 'lime', 0.8: 'yellow', 1.0: 'red'}).add_to(m_heat)
        
        print(f"Heatmap: {len(heat_data):,} points")
        display(m_heat)
    else:
        print("Aucune donnée valide après nettoyage des NaN")
else:
    print("Aucune donnée")

### 6.4 Corrélation: Latitude vs Couverture Végétale

Y a-t-il une corrélation entre la latitude (zones géographiques) et la végétation ?

In [ ]:
query_veg = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "vegetation_cover_high" or r["_field"] == "vegetation_cover_low" or r["_field"] == "location-lat")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> limit(n: 1000)
'''

result_veg = query_api.query_data_frame(query_veg, org=INFLUX_ORG)
df_veg = process_flux_result(result_veg)

if df_veg is not None and len(df_veg) > 0:
    # Nettoyer les données en supprimant les lignes avec des NaN
    df_veg_clean = df_veg.dropna(subset=['location-lat', 'vegetation_cover_high', 'vegetation_cover_low'])
    
    if len(df_veg_clean) > 0:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        ax1.scatter(df_veg_clean['location-lat'], df_veg_clean['vegetation_cover_high'], 
                    alpha=0.3, s=2, color='#2E7D32')
        ax1.set_xlabel('Latitude (°)', fontsize=11)
        ax1.set_ylabel('Haute Végétation', fontsize=11)
        ax1.set_title('Latitude vs Haute Végétation', fontsize=13, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        
        corr_high = df_veg_clean['location-lat'].corr(df_veg_clean['vegetation_cover_high'])
        ax1.text(0.05, 0.95, f'Corr: {corr_high:.3f}', transform=ax1.transAxes, 
                 fontsize=10, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        ax2.scatter(df_veg_clean['location-lat'], df_veg_clean['vegetation_cover_low'], 
                    alpha=0.3, s=2, color='#F57C00')
        ax2.set_xlabel('Latitude (°)', fontsize=11)
        ax2.set_ylabel('Basse Végétation', fontsize=11)
        ax2.set_title('Latitude vs Basse Végétation', fontsize=13, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        corr_low = df_veg_clean['location-lat'].corr(df_veg_clean['vegetation_cover_low'])
        ax2.text(0.05, 0.95, f'Corr: {corr_low:.3f}', transform=ax2.transAxes,
                 fontsize=10, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        plt.tight_layout()
        plt.show()
    else:
        print("Aucune donnée valide")
else:
    print("Aucune donnée")

## Fin du TP : Récapitulatif

### Ce que nous avons appris :

Concepts Time-Series :
- Différence entre bases relationnelles, documents, et time-series
- Pourquoi InfluxDB pour le tracking GPS et les métriques

Schema Design InfluxDB :
- Tags vs Fields : la décision la plus critique
- Impact de la cardinalité sur les performances
- Anti-patterns à éviter (tags constants, haute cardinalité)

Langage Flux :
- Requêtes basiques (`from`, `range`, `filter`)
- Agrégations (`count`, `mean`, `group`)
- Window functions pour downsampling
- Pivot pour restructurer les données

Analyse Time-Series :
- Requêtes pour trouver les extremes (min/max)
- Détection de patterns saisonniers avec agrégation temporelle
- Analyse comparative (été vs hiver)

Optimisation & Gestion :
- Retention policies pour la gestion du cycle de vie des données
- Compression et efficacité du stockage
- Impact du schema design sur les performances

Visualisations (Démonstration) :
- Cartes interactives de migration
- Time-series plots
- Heatmaps de densité
- Analyse de corrélation

### InfluxDB vs MongoDB : Quand utiliser quoi ?

| Critère | InfluxDB | MongoDB |
|---------|----------|---------|
| Use case | Métriques, IoT, tracking temporel | Documents flexibles, applications web |
| Structure | Tags + Fields + Timestamp | JSON documents |
| Queries temporelles | Ultra-rapides | Correct avec indexes |
| Compression | 10-20x | Moyenne |
| Flexibilité schéma | Tags/Fields fixes | Total freedom |
| Relations complexes | Pas adapté | Avec $lookup |

Choix pour notre projet :
- InfluxDB : données temporelles, tracking GPS, métriques environnementales
- MongoDB : aurait fonctionné mais moins efficace pour les requêtes temporelles

### Prochaines étapes suggérées :

1. Continuous Queries : Pré-calculer des agrégations (downsampling automatique)
2. Tasks : Automatiser le traitement de données
3. Alerting : Détecter des patterns anormaux (oiseau hors zone attendue)
4. Grafana : Dashboards temps-réel pour monitoring
5. Sharding & Replication : Scalabilité horizontale pour très gros volumes

Ressources :
- [Documentation InfluxDB](https://docs.influxdata.com/influxdb/v2.0/)
- [Flux Language Guide](https://docs.influxdata.com/flux/v0.x/)
- [InfluxDB University](https://university.influxdata.com/)